# Part 1: Load Data

In [1]:
#the module 'sys' allows istalling module from inside Jupyter
import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

!{sys.executable} -m pip install sklearn
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer 
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt
%matplotlib inline

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liuxinran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/liuxinran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/liuxinran/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/liuxinran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [2]:
df=pd.read_excel('Ford_data.xlsx')
df.dropna(subset=['text'],inplace=True)
data=df.loc[:,'text'].tolist()
normalized_data = normalize_corpus(data) 
normalized_data


/Users/liuxinran/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


['might read previous post ford taurus might right wit end take ford taurus se shop warranty work vehicle mile quality use job ford least concern right warranty issue ford display car stereo blink radio work fine far sound visual display short something think take fix simple task unfortunately wrong keep car day change radio new ford require send old radio factory repair end eighth day dealership contact car ready pick ahead pick car sit driver seat put key radio display radio fact display sound work radio piece dead hardware immediately service advisor ford check radio actually work send radio answer stupid ford think thing well ford overnighted radio repair overnighted radio call today pick car turn radio car seem run fine fifteen minute dealership radio die return dealership send radio ford ford work inquire new radio figure car month old ford policy work radio many time need problem resolve meanwhile rent car day dime ford provide loaner unless purchase extended warranty ford keep 

# Part 2: Tokenizing and Stemming

In [3]:
stopwords=nltk.corpus.stopwords.words('english')
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") # html br 
stopwords.append("car")
stopwords.append('ford')
stopwords.append('Ford')

In [4]:
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer('english')
def tokenization_and_stem(text):
    tokens=[]
    for i in range(len(text)):
        tokens.append(nltk.word_tokenize(text[i]))
    res=[]
    for token in tokens:
        cur_token=[i.lower()for i in token]
        temp=[i for i in cur_token if i not in stopwords]
        stem=[stemmer.stem(t) for t in temp]
        res.append(stem)
    return tokens
            

In [5]:
new_data=tokenization_and_stem(normalized_data)
new_data
new_list=[]
for i in new_data:
    new_list.append(' '.join(i))
new_list       

['might read previous post ford taurus might right wit end take ford taurus se shop warranty work vehicle mile quality use job ford least concern right warranty issue ford display car stereo blink radio work fine far sound visual display short something think take fix simple task unfortunately wrong keep car day change radio new ford require send old radio factory repair end eighth day dealership contact car ready pick ahead pick car sit driver seat put key radio display radio fact display sound work radio piece dead hardware immediately service advisor ford check radio actually work send radio answer stupid ford think thing well ford overnighted radio repair overnighted radio call today pick car turn radio car seem run fine fifteen minute dealership radio die return dealership send radio ford ford work inquire new radio figure car month old ford policy work radio many time need problem resolve meanwhile rent car day dime ford provide loaner unless purchase extended warranty ford keep 

# Part 3 LDA 

In [6]:
bow_vectorizer = CountVectorizer()
bow_data=bow_vectorizer.fit_transform(new_list)
pd.DataFrame(data = bow_data.todense(), columns = bow_vectorizer.get_feature_names())

,000k,000km,000kms,00a,00p,0l,0p,10000lbs,1000lb,100hp,...,zoom,zr,ztec,ztech,zts,ztw,zx,zx2,zx3,zx5
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
lda = LatentDirichletAllocation(n_components=4, max_iter=500,
                                           doc_topic_prior = 0.5,
                                           topic_word_prior = 0.5).fit(bow_data)


In [8]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

In [9]:
no_top_words = 40
display_topics(lda, bow_vectorizer.get_feature_names(), no_top_words)

Topic 0:
truck seat vehicle drive ford like explorer good look suv well door power rear road br van use new wheel need front much great car even quot want ride large room big time thing engine little buy gas take expedition
Topic 1:
car ford drive seat like good look engine mustang well power taurus much model even little rear quot wheel great speed feel year interior buy really front use control small think driver new door trunk time lot road br focus
Topic 2:
lightning 454ss roadiem lightening usa missouri damn australian cassete write rod twenty journey victoria australia 150s waist collapse sierra eleven fiesta thats urge underdrive s10 primarily rally aussi mike ladycynic juliette seether166 imames mtuairau emeleel theworm sherrylee tiffer0220 diverpam wolfman309
Topic 3:
car ford problem mile time buy replace year drive br vehicle new transmission take repair work fix month engine first quot start dealer warranty brake day another service purchase good still like never know light

In [10]:
topic_keywords = get_topic_words(vectorizer=bow_vectorizer, lda_model=lda, n_words=40)  
df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,...,Word 30,Word 31,Word 32,Word 33,Word 34,Word 35,Word 36,Word 37,Word 38,Word 39
Topic 0,truck,seat,vehicle,drive,ford,like,explorer,good,look,suv,...,room,big,time,thing,engine,little,buy,gas,take,expedition
Topic 1,car,ford,drive,seat,like,good,look,engine,mustang,well,...,think,driver,new,door,trunk,time,lot,road,br,focus
Topic 2,lightning,454ss,roadiem,lightening,usa,missouri,damn,australian,cassete,write,...,theworm,wolfman309,technologyrep,teddiec,elorraine,emeleel,hnnygrlca,mtuairau,hirohito99,ginahill
Topic 3,car,ford,problem,mile,time,buy,replace,year,drive,br,...,still,like,never,know,light,need,thing,dealership,bad,well


In [11]:
word_weights = lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
word_weights_df = pd.DataFrame(word_weights.T, 
                               index = bow_vectorizer.get_feature_names(), 
                               columns = ["Topic_" + str(i) for i in range(4)])
word_weights_df.head(10)

,Topic_0,Topic_1,Topic_2,Topic_3
000k,0.000007,0.000006,0.000071,0.000040
000km,0.000005,0.000005,0.000070,0.000044
000kms,0.000005,0.000005,0.000070,0.000031
00a,0.000005,0.000005,0.000071,0.000019
00p,0.000005,0.000005,0.000071,0.000019
0l,0.000103,0.000302,0.000079,0.000012
0p,0.000005,0.000005,0.000070,0.000018
10000lbs,0.000014,0.000005,0.000071,0.000006
1000lb,0.000005,0.000013,0.000071,0.000007
100hp,0.000005,0.000005,0.000071,0.000018


In [12]:
word_weights_df.sort_values(by='Topic_0',ascending=False).head(50)

,Topic_0,Topic_1,Topic_2,Topic_3
truck,0.016221,0.000006,0.000074,0.000749
seat,0.012002,0.009164,0.000072,0.000160
vehicle,0.011323,0.002530,0.000078,0.007011
drive,0.011084,0.009699,0.000074,0.007439
ford,0.010656,0.009711,0.000074,0.018276
like,0.008526,0.008476,0.000073,0.003907
explorer,0.007180,0.000005,0.000071,0.000420
good,0.006084,0.007572,0.000073,0.003953
look,0.005891,0.007353,0.000074,0.002536
suv,0.005866,0.000124,0.000071,0.000009


In [13]:
#prepare to display result in the Jupyter notebook
pyLDAvis.enable_notebook()

#run the visualization [mds is a function to use for visualizing the "distance" between topics]
pyLDAvis.sklearn.prepare(lda, bow_data, bow_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      269.562408 -124.803398       1        1  37.402334
0        1.782994 -295.079681       2        1  36.032096
3      220.811432 -343.832123       3        1  26.037692
2       50.533993  -76.050934       4        1   0.527877, topic_info=            Term         Freq        Total Category  logprob  loglift
1961         car  6808.000000  6808.000000  Default  30.0000  30.0000
11570      truck  1684.000000  1684.000000  Default  29.0000  29.0000
8587     problem  1565.000000  1565.000000  Default  28.0000  28.0000
9206     replace   654.000000   654.000000  Default  27.0000  27.0000
4233    explorer   752.000000   752.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
12532      write     0.870303   116.981065   Topic4  -7.4333   0.3431
11174      thats     0.636141    26.723167   Topic4  -7.7468   1.5062
9536         s10     0.550820     6.466083   Topic4  -7.8908   2.7811
8870       rally     0.528458     3.754423   Topic4  -7.9322   3.2833
8555   primarily     0.547386    15.802848   Topic4  -7.8970   1.8813

[292 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
68         1  0.040819   146s
68         2  0.952442   146s
68         3  0.006803   146s
69         1  0.090787   146t
69         2  0.897784   146t
...      ...       ...    ...
12625      1  0.949637  zetec
12625      2  0.031655  zetec
12625      3  0.031655  zetec
12641      1  0.980897    zx2
12641      3  0.014640    zx2

[614 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])

# Part 4 : Vader lexincon-based sentiment analysis 

In [14]:
from sklearn import model_selection
cleaned=pd.DataFrame(new_list,columns=['cleaned_text'])
cleaned.index=df.index
combine_df=pd.concat([df,cleaned],axis=1)
# test train data spilt 
y=combine_df['class']
X=combine_df['cleaned_text']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=1)

In [15]:
list_x_train=X_train.tolist()
list_x_test=X_test.tolist()

In [16]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer  


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/liuxinran/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [17]:
def sentiment(sentences):
    analyzer = SentimentIntensityAnalyzer() 
    res=[]
    for sentence in sentences:
        score=analyzer.polarity_scores(sentence)
        res.append(score)
    return res

In [18]:
sentiment_train=sentiment(list_x_train)
sentiment_train

[{'neg': 0.105, 'neu': 0.708, 'pos': 0.187, 'compound': 0.9005},
 {'neg': 0.146, 'neu': 0.613, 'pos': 0.241, 'compound': 0.9441},
 {'neg': 0.108, 'neu': 0.802, 'pos': 0.09, 'compound': -0.3506},
 {'neg': 0.142, 'neu': 0.736, 'pos': 0.121, 'compound': -0.0516},
 {'neg': 0.213, 'neu': 0.718, 'pos': 0.07, 'compound': -0.9287},
 {'neg': 0.247, 'neu': 0.632, 'pos': 0.121, 'compound': -0.8905},
 {'neg': 0.085, 'neu': 0.832, 'pos': 0.083, 'compound': 0.1764},
 {'neg': 0.0, 'neu': 0.563, 'pos': 0.437, 'compound': 0.9834},
 {'neg': 0.088, 'neu': 0.704, 'pos': 0.208, 'compound': 0.9917},
 {'neg': 0.109, 'neu': 0.691, 'pos': 0.2, 'compound': 0.999},
 {'neg': 0.07, 'neu': 0.722, 'pos': 0.208, 'compound': 0.9136},
 {'neg': 0.066, 'neu': 0.652, 'pos': 0.282, 'compound': 0.995},
 {'neg': 0.076, 'neu': 0.595, 'pos': 0.329, 'compound': 0.9907},
 {'neg': 0.065, 'neu': 0.688, 'pos': 0.247, 'compound': 0.991},
 {'neg': 0.214, 'neu': 0.65, 'pos': 0.136, 'compound': -0.9968},
 {'neg': 0.175, 'neu': 0.763, '

In [19]:
# gain accuracy 
sentiment_test=sentiment(list_x_test)
sentiment_test=pd.DataFrame(sentiment_test)

### set new lable for sentiment 

In [20]:
def score(score):
    if score>=0.7:
        return "Pos"
    elif 0.4<score<0.7:
        return "Neu"
    else:
        return "Neg"
sentiment_test['sentiment']=sentiment_test.compound.apply(score)
sentiment_test

,neg,neu,pos,compound,sentiment
0,0.143,0.675,0.182,0.9781,Pos
1,0.079,0.682,0.239,0.9991,Pos
2,0.158,0.683,0.159,0.6396,Neu
3,0.054,0.755,0.190,0.8612,Pos
4,0.083,0.675,0.242,0.9611,Pos
...,...,...,...,...,...
341,0.113,0.588,0.299,0.9948,Pos
342,0.136,0.746,0.118,-0.6204,Neg
343,0.105,0.791,0.104,-0.2924,Neg
344,0.027,0.718,0.256,0.9882,Pos


### Vador Lexicon accuracy

In [21]:
test_polarity=np.array(y_test)
from sklearn import metrics
print('Accuracy Rate:', np.round(metrics.accuracy_score(test_polarity, 
                                 sentiment_test['sentiment']), 3),"\n")

Accuracy Rate: 0.636 



### Predicted sentiment with original label 

In [22]:
sentiment_df=sentiment(new_list)
sentiment_cust=pd.DataFrame(sentiment_df)
sentiment_cust['sentiment']=sentiment_cust.compound.apply(score)
sentiment_cust.index=df.index
final=pd.concat([df,sentiment_cust],axis=1)

### topic pick 

In [23]:
topic_words=df_topic_words.T
topic0=topic_words['Topic 0'].tolist()
topic1=topic_words['Topic 1'].tolist()
topic2=topic_words['Topic 2'].tolist()
topic3=topic_words['Topic 3'].tolist()
topic0

['truck',
 'seat',
 'vehicle',
 'drive',
 'ford',
 'like',
 'explorer',
 'good',
 'look',
 'suv',
 'well',
 'door',
 'power',
 'rear',
 'road',
 'br',
 'van',
 'use',
 'new',
 'wheel',
 'need',
 'front',
 'much',
 'great',
 'car',
 'even',
 'quot',
 'want',
 'ride',
 'large',
 'room',
 'big',
 'time',
 'thing',
 'engine',
 'little',
 'buy',
 'gas',
 'take',
 'expedition']

### normalized review data join with original DataFrame

In [24]:
table=pd.DataFrame(new_list,columns=['new_text'])
table.index=final.index
new_data=pd.concat([table,final],axis=1)
new_data

,new_text,class,text,neg,neu,pos,compound,sentiment
0,might read previous post ford taurus might rig...,Pos,Some of you might have read my previous post ...,0.085,0.830,0.085,-0.4588,Neg
1,buy new ford taurus month ago love former car ...,Pos,I bought my new 1999 Ford Taurus just two mon...,0.100,0.632,0.268,0.9896,Pos
2,first car ford old clunky ugly sucker run like...,Pos,My first car was a Ford One of those old cl...,0.147,0.635,0.217,0.8625,Pos
3,last spring new car ford taurus happy great de...,Pos,Last spring we got a new car a 1999 ford tau...,0.000,0.558,0.442,0.9892,Pos
4,recently buy used ford taurus model v6 engine ...,Pos,Recently I bought a used 99 Ford Taurus The...,0.070,0.690,0.240,0.9887,Pos
...,...,...,...,...,...,...,...,...
1377,good car great concept faulty execution okay l...,Neg,It s a good car a great concept and a fault...,0.076,0.733,0.191,0.8720,Pos
1378,family always buy ford car truck always treat ...,Neg,Our families have always bought Ford cars and...,0.095,0.721,0.184,0.7216,Pos
1379,victim ford focus zts tragic pleasant car driv...,Neg,I am the victim of a 2001 Ford Focus ZTS It ...,0.204,0.601,0.196,0.0772,Neg
1380,purchase ford probe se april mile odometer mil...,Neg,I purchased a 1994 Ford Probe SE in April 199...,0.070,0.709,0.221,0.9864,Pos


# Part 5: Topic in Pos/Neg/Neu reviews

In [49]:
# sentiment split 
pos=new_data.loc[new_data.sentiment=='Pos',['new_text','sentiment']]
neg=new_data.loc[new_data.sentiment=='Neg',['new_text','sentiment']]
neu=new_data.loc[new_data.sentiment=='Neu',['new_text','sentiment']]
pos_lst=pos['new_text'].tolist()
neg_lst=neg['new_text'].tolist()
neu_lst=neu['new_text'].tolist()
def split_word(text):
    word=[]
    for line in text:
        word.append(line.strip('\n').split())
    return word

def topic_choice(texts,topic_words):
    topic=[]
    text=split_word(pos_lst)
    for word in range(len(text)):
        count=0
        for i in text[word]:
            if i.lower() in topic_words:
                count+=1
        topic.append([word,count])
    return topic
# postivie topic 
topic0_pos_choice=topic_choice(pos,topic0)
topic1_pos_choice=topic_choice(pos,topic1)
topic2_pos_choice=topic_choice(pos,topic2)
topic3_pos_choice=topic_choice(pos,topic3)
# negative topic 
topic0_neg_choice=topic_choice(neg,topic0)
topic1_neg_choice=topic_choice(neg,topic1)
topic2_neg_choice=topic_choice(neg,topic2)
topic3_neg_choice=topic_choice(neg,topic3)
# neutral topic
topic0_neu_choice=topic_choice(neu,topic0)
topic1_neu_choice=topic_choice(neu,topic1)
topic2_neu_choice=topic_choice(neu,topic2)
topic3_neu_choice=topic_choice(neu,topic3)

# count topic 
sumv=0
for value,count in topic3_pos_choice:
    if count>=10:
        sumv+=1
print(sumv)
        

1028


### Topic review DataFrame

In [50]:
output=[[0.99,0.97,0.04,0.96],
        [0.98,0.92,0.0,0.92],
        [0.97,0.95,0.0,0.99]]
topic_ratio=pd.DataFrame(output,index=['Pos','Neu','Neg'],columns=['Topic0','Topic1','Topic2','Topic3'])
topic_ratio

,Topic0,Topic1,Topic2,Topic3
Pos,0.99,0.97,0.04,0.96
Neu,0.98,0.92,0.00,0.92
Neg,0.97,0.95,0.00,0.99


# Part 6:Try other classifier methods to check customize label accuracy (supervised learning)  

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range = (1,2)) 
feature_matrix_TRAIN = vectorizer.fit_transform(list_x_train).astype(float)

In [29]:
feature_matrix_TRAIN_names = vectorizer.get_feature_names() 
feature_matrix_TRAIN_table = pd.DataFrame(data = feature_matrix_TRAIN.todense(), 
                                          columns = feature_matrix_TRAIN_names)
feature_matrix_TRAIN_table.head()

,000k,000k fairly,000k mile,000km,000km think,000km transmission,000km warranty,000kms,000kms 52500miles,000kms odometer,...,zx3 test,zx3 unfortunately,zx3 win,zx3 wrong,zx5,zx5 conventional,zx5 liter,zx5 sedan,zx5 state,zx5 stick
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
feature_matrix_TEST = vectorizer.transform(list_x_test)

### Using Support Vector Machines(SVM) to train our custom sentiment classifier

In [51]:
from sklearn import linear_model
train_polarity=np.array(y_train)
svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(feature_matrix_TRAIN, train_polarity)
predicted_svm = svm.predict(feature_matrix_TEST)

In [32]:
print('Accuracy rate:', np.round(metrics.accuracy_score(test_polarity, predicted_svm), 3))

Accuracy rate: 0.844
